In [3]:
using NBInclude;
using Plots; default(fmt=:png,ls=:auto);
@nbinclude("deal_features.ipynb");

human_readable (generic function with 1 method)

In [1]:
function bid!(Q,deal,database_state,database_target,epsilon_greedy)
    bidding_hist=zeros(36,1);
    instance=(deal[1],deal[2],bidding_hist);
    for t in 1:turn_depth
        if t==1
            current_bid=1;
            state=atype(instance[(t-1)%2+1]);
            target=determine_target(deepcopy(deal[3][:,(t-1)%2+1]),current_bid); #Algorithm P7
            target[1]=0.1
        else
            b=findlast(x->x==1,bidding_hist);
            current_bid=b[1];
            state=atype(vcat(instance[(t-1)%2+1],bidding_hist));  
            target=determine_target(deepcopy(deal[3][:,(t-1)%2+1]),current_bid); #Algorithm P
        end


        #exploration
        action,_=explore(Q[t],state,current_bid,epsilon_greedy);
        #Update bidding history

        bidding_hist[action]=1;
        #Data for experience replay

        
        
        # current bid is pass----- 

        if action== current_bid
            target=target+deepcopy(deal[3][:,(t-1)%2+1]);
            push!(database_state[t],state);
            push!(database_target[t],target);  
            break;
            
        end
                  
        push!(database_state[t],state);
        push!(database_target[t],target);  
    end   

end

function bid_to_the_end(Q,deal_set,number_of_deals)
    database_state=[[],[],[],[],[],[]];
    database_target=[[],[],[],[],[],[]];
    for i in 1:number_of_deals
        bid!(Q,deal_set[rand(1:end)],database_state,database_target,0);
    end
    return database_state, database_target;
end

bid_to_the_end (generic function with 1 method)

In [1]:
function bid(Q,deal;debug=false)
    bidding_hist=zeros(36,1);
    instance=(deal[1],deal[2],bidding_hist);
    q_values_data=[];
    target_data=[];
    for t in 1:turn_depth
        if debug
            println(string(t," bid"));
        end
        if t==1
            current_bid=1;
            state=atype(instance[(t-1)%2+1]);
            target=determine_target(deepcopy(deal[3][:,(t-1)%2+1]),current_bid); #Algorithm P7
            target[1]=0.1
        else
            b=findlast(x->x==1,bidding_hist);
            current_bid=b[1];
            state=atype(vcat(instance[(t-1)%2+1],bidding_hist));  
            target=determine_target(deepcopy(deal[3][:,(t-1)%2+1]),current_bid); #Algorithm P
        end
        
        #Data for experience replay
        #exploration
        action,q_values=explore(Q[t],state,current_bid,0);
        # current bid is pass----- 
        if action== current_bid
            target=target+deepcopy(deal[3][:,(t-1)%2+1]);
            if debug
                print("Bid"); print("\t"); print("Q value"); print("\t \t"); print("Cost");  print("\t"); println("Loss")
                for i in 1:36
                    print(bids[i]); print("\t"); print(q_values[i]);  print("\t"); print(target[i]); print("\t"); println(q_values[i]-target[i])
                end
            push!(q_values_data,q_values)
            push!(target_data,target)
            end
            break;
        end
        if debug
            print("Bid"); print("\t"); print("Q value"); print("\t \t"); print("Cost");  print("\t"); println("Loss")
            for i in 1:36
                print(bids[i]); print("\t"); print(q_values[i]);  print("\t"); print(target[i]); print("\t"); println(q_values[i]-target[i])
            end
            push!(q_values_data,q_values)
            push!(target_data,target)
        end    
        
        #Update bidding history
        bidding_hist[action]=1;
    end   

    return deal[1],deal[2],bidding_hist,q_values_data,target_data
end

bid (generic function with 1 method)